In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeledTrainData.tsv'
UNLABELED_TRAIN_DF = BASE_DIR + 'unlabeledTrainData.tsv'
TEST_DF = BASE_DIR + 'testData.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sentences_for_word2vec_convert_10.csv', 'cc.en.300.vec', 'sampleSubmission.csv', 'fasttext_embed_pretrained_cc_convert_10.sh', 'labeled_train_clean_reviews.csv', 'word2vec_model_300dim_40minwords_10context_stemmed', 'labeledTrainData.tsv', 'sentences_for_word2vec.csv', 'fasttext_300features_40minwords_10context.vec', 'labeled_train_clean_reviews_stemmed.csv', 'fasttext_300features_40minwords_10context_pretrained_cc.vec', 'test_clean_reviews_convert_10.csv', 'fasttext_300features_40minwords_10context_convert_10.vec', 'fasttext_300features_40minwords_10context.bin', 'test_submission.csv', 'fasttext_embed_pretrained_wiki.sh', 'fasttext_300features_40minwords_10context_pretrained_wiki_convert_10.bin', 'wiki-news-300d-1M.vec', 'fasttext_embed_pretrained_wiki_convert_10.sh', 'testData.tsv', 'fasttext_300features_40minwords_10context_pretrained_wiki.bin', 'cc.en.300.bin', 'fasttext_300features_40minwords_10context_pretrained_wiki.vec', 'fasttext

In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
unlabeled_train = pd.read_csv(UNLABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
test = pd.read_csv(TEST_DF, header = 0, delimiter = '\t', quoting=3)
print "Read %d labeled train reviews, %d unlabeled train reviews, " \
          "and %d test reviews" % (labeled_train["review"].size, unlabeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviews, 50000 unlabeled train reviews, and 25000 test reviews


Data leakage

Check if test["sentiment"] is correct 

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

Credits: Kaggle tutorial

In [4]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def review_to_clean_review(review, remove_stopwords=False, remove_numbers=True, stem_words=False):
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    
    if remove_numbers:
        review_text = re.sub("[0-9]", " ", review_text)
    else:
        review_text = review_text.replace('0', ' zero ')
        review_text = review_text.replace('1', ' one ')
        review_text = review_text.replace('2', ' two ')
        review_text = review_text.replace('3', ' three ')
        review_text = review_text.replace('4', ' four ')
        review_text = review_text.replace('5', ' five ')
        review_text = review_text.replace('6', ' six ')
        review_text = review_text.replace('7', ' seven ')
        review_text = review_text.replace('8', ' eight ')
        review_text = review_text.replace('9', ' nine ')
    
    review_text = review_text.lower()
    
    if remove_stopwords:
        words = review_text.split()
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        review_text = " ".join(words)
    
    if stem_words:
        words = review_text.split()
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(word) for word in words]
        review_text = " ".join(words)
    
    return(review_text.strip())

def review_to_wordlist(review, remove_stopwords=False, remove_numbers=True, stem_words=False):

    words = review_to_clean_review(review, remove_stopwords, remove_numbers, stem_words).split()
    return words

def review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=True, stem_words=False):
    
    raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords, remove_numbers, stem_words))
    return sentences

In [5]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
example_review = "This is an example of a movie review. It has two sentences."
print(review_to_sentences(example_review, tokenizer, remove_numbers=False, stem_words=True))

[[u'this', u'is', u'an', u'exampl', u'of', u'a', u'movi', u'review'], [u'it', u'has', u'two', u'sentenc']]


In [6]:
%%script false 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Initialize an empty list of sentences
stemmed_sentences = []
sentences = []
# Parsing sentences from training set
counter = 0.
for review in labeled_train["review"]:
#     stemmed_sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=True)
    sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=False)
    if counter % 5000. == 0.:
        print "Labled train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

counter = 0.
# Parsing sentences from unlabeled set
for review in unlabeled_train["review"]:
#     stemmed_sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=True)
    sentences += review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=False, stem_words=False)
    if counter % 5000. == 0.:
        print "Unlabled train review %d of %d" % (counter, len(unlabeled_train["review"]))
    counter = counter + 1.

In [7]:
%%script false 
print(np.random.choice(sentences, 10))

In [8]:
%%script false 
joined_sentences = [" ".join(sentence).strip() for sentence in sentences]
output_sentences = pd.DataFrame(data={"sentences": joined_sentences})
output_sentences.to_csv(os.path.join('../', 'input', "sentences_for_word2vec.csv"), index=False)

In [9]:
labeled_train_clean_reviews = []
counter = 0.
for review in labeled_train["review"]:
    clean_review = review_to_clean_review(review, remove_stopwords=True, remove_numbers=False, stem_words=False)
    labeled_train_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

test_clean_reviews = []
counter = 0.
for review in test["review"]:
    clean_review = review_to_clean_review(review, remove_stopwords=True, remove_numbers=False, stem_words=False)
    test_clean_reviews.append(clean_review)
    if counter % 5000. == 0.:
        print "Test review %d of %d" % (counter, len(test["review"]))
    counter = counter + 1.

Train review 0 of 25000
Train review 5000 of 25000
Train review 10000 of 25000
Train review 15000 of 25000
Train review 20000 of 25000
Test review 0 of 25000
Test review 5000 of 25000
Test review 10000 of 25000
Test review 15000 of 25000
Test review 20000 of 25000


In [10]:
print(np.random.choice(labeled_train_clean_reviews, 10))
print(np.random.choice(test_clean_reviews, 10))

[u'cynical writers pander may american citizen need see americans pat back hour half order enjoy film astonished commented utterly jingoistic sentimental trite dialogue film really historical inaccuracies film gross offensive u five seven one changes british submariners american ones still walk away feeling little slimy really germans film nothing admire americans battle scenes sad unnecessary like say writers obvious many us capture real people real problems real pressure taken well tested shortcuts lucky always nationalist nutjobs appreciate sugar coated tripe'
 u'brilliant acting lesley ann warren best dramatic hobo lady ever seen love scenes clothes warehouse second none corn face classic good anything blazing saddles take lawyers also superb accused turncoat selling boss dishonest lawyer pepto bolt shrugs indifferently lawyer says three funny words jeffrey tambor favorite later larry sanders show fantastic mad millionaire wants crush ghetto character malevolent usual hospital scen

In [11]:
output_labeled_train_clean_reviews = \
    pd.DataFrame(data={"id": labeled_train["id"], "sentiment": labeled_train["sentiment"], "review": labeled_train_clean_reviews})
output_test_clean_reviews = \
    pd.DataFrame(data={"id": test["id"], "review": test_clean_reviews})

output_labeled_train_clean_reviews.to_csv(os.path.join('../', 'input', "labeled_train_clean_reviews_remove_stopwords.csv"), index=False, quoting=3)
output_test_clean_reviews.to_csv(os.path.join('../', 'input', "test_clean_reviews_remove_stopwords.csv"), index=False, quoting=3)

Training the Word2Vec model

In [12]:
%%script false 
#Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context_" +\
    "new"

print "Training the Word2Vec model..."
model = Word2Vec(sentences, workers=NUM_THREADS, \
                 size=EMBEDDING_DIM, min_count=MIN_WORD_COUNT, \
                 window=CONTEXT, sample=DOWNSAMPLING, seed=1)
model.save(WORD2VEC_MODEL_FILE)


Generate the txt file for FastText

In [13]:
%%script false 
sentences = pd.read_csv("../input/sentences_for_word2vec.csv", header = 0)
# sentences_stemmed = pd.read_csv("../input/sentences_for_word2vec_stemmed.csv", header = 0)
sentences["sentences"] = sentences["sentences"].astype(str)
sentences = sentences["sentences"].tolist()
# sentences_stemmed = sentences_stemmed["sentences"].tolist()
print(np.random.choice(sentences, 10))

In [14]:
%%script false 
output_sentences = open('../input/sentences_for_fasttext.txt', 'w')
for sentence in sentences:
    output_sentences.write(sentence + '\n')
output_sentences.close()